In [1]:
from helper_fun import get_evaluator_info, evaluate_circ, apply_measurement
from qcg.generators import gen_supremacy, gen_hwea

circ = gen_supremacy(2,2,8)
device_name = 'ibmq_boeblingen'
evaluator_info = get_evaluator_info(circ=circ,device_name=device_name,fields=['num_shots'])
qasm_noiseless_fc = evaluate_circ(circ=circ,backend='noiseless_qasm_simulator',evaluator_info=evaluator_info)

Generating 2x2, 1+8+1 supremacy circuit
type of counts: <class 'dict'>
type of counts: <class 'dict'>
type of counts: <class 'dict'>
